In [1]:
# EXAONE 
# 디코더기반 트랜스포머 모델 -> 자기회귀 모델 -> 토큰을 매번 재계산하는 문제 -> 어텐션층에서 키와 값을 캐시에 저장하고 재사용하는 기법등장 -> 캐시 크기가 커지는 문제
# 멀티 쿼리 어텐션 -> 멀티헤드어텐션에서 키와 값을 모든 헤드에서 공유하는 방식
# 그룹 쿼리 어텐션 -> 모든 헤드에서 키와 값을 공유하지 않고, 몇 개의 헤드씩 나눠서 공유하는 방식 -> EXAONE 
# 활성화함수로 실루함수 사용
# RMS 정규화 사용 -> 기존의 정규화 방식에서 평균을 사용하지않는 방식 -> 계산속도빠름

In [2]:
# EXAONE-3.5의 24억 파라미터 버전 사용
# 채팅 템플릿을 활용할 때 좋은 결과를 얻을 수 있음
# 토크나이저를 명시적으로 로드하여 pipeline() 함수에 전달 가능

# EXAONE 토크나이저 로드
from transformers import AutoTokenizer

exaone_tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct")

/home/parkwooyeol/workspace/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import pipeline

pipe = pipeline(task="text-generation", model="LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct", tokenizer=exaone_tokenizer, device=0, trust_remote_code=True)
# trust_remote_code -> 트랜스포머 패키지의 코드 외의 독자적인 코드로 모델을 정의 가능


2025-09-04 11:32:17.534678: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-04 11:32:17.672842: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-04 11:32:17.672870: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-04 11:32:17.673498: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-04 11:32:17.720156: I tensorflow/core/platform/cpu_feature_g

In [4]:
# 채팅 템플릿 만들기
messages = [
    {"role": "system",
     "content": "너는 쇼핑몰 홈페이지에 올라온 질문에 대답하는 Q&A 챗봇이야.\
      확정적인 답변을 하지 말고 제품 담당자가 정확한 답변을 하기 위해 \
      시간이 필요하다는 간단하고 친절한 답변을 생성해줘."},
    {"role": "user", "content":"이 다이어리에 내년도 공휴일이 표시되어 있나요?"}
]  

In [5]:
# 파이프 라인 객체호출
pipe(messages, max_new_tokens=200)

[{'generated_text': [{'role': 'system',
    'content': '너는 쇼핑몰 홈페이지에 올라온 질문에 대답하는 Q&A 챗봇이야.      확정적인 답변을 하지 말고 제품 담당자가 정확한 답변을 하기 위해       시간이 필요하다는 간단하고 친절한 답변을 생성해줘.'},
   {'role': 'user', 'content': '이 다이어리에 내년도 공휴일이 표시되어 있나요?'},
   {'role': 'assistant',
    'content': '물론 궁금하시군요! 다이어리의 공휴일 표시 내용은 정확하게 확인해 드리기 위해 제품 담당자님께 잠시 확인해 볼게요. 빠르게 답변드릴 수 있도록 노력하겠습니다. 혹시 다른 질문이 있으시다면 편하게 말씀해 주세요! 😊'}]}]

In [6]:
# role이 assistant인 항목 추가 content키에 담긴 내용이 EXAONE이 만든 답변

# 모델이 생성한 텍스트만 출력
pipe(messages, max_new_tokens=500, return_full_text=False)

[{'generated_text': '안녕하세요! 다이어리의 공휴일 정보에 대해 궁금해하시는군요. 정확한 답변을 드리기 위해서는 다이어리의 제작 연도와 종류를 확인해야 합니다. 공휴일 정보는 해당 연도의 국가별 공휴일 정책에 따라 다르기 때문이에요. 혹시 제품에 대한 추가적인 정보나 궁금한 점이 있으시다면 알려주시면 더욱 구체적인 답변을 드릴 수 있을 것 같아요. 잠시 기다리고 계시면 담당자가 가장 빠르게 도와드릴 수 있도록 하겠습니다! 감사합니다.'}]

In [7]:
# 확률적으로 토큰을 선택하고 싶다면 do_sample 매개변수를 True로 지정
# 키값만 출력
output = pipe(messages, max_new_tokens=200, return_full_text=False, do_sample=True)
print(output[0]['generated_text'])

안녕하세요! 다이어리에 내년도 공휴일 정보가 포함되어 있는지에 대해 자세히 알려드리기 위해서는 제품 설명서나 해당 다이어리의 특정 모델 정보가 필요할 것 같아요. 현재로선 정확한 답변을 드리기 어렵습니다. 제품 담당자님께 문의하시거나, 제품 페이지의 FAQ 섹션을 확인해 보시는 것도 좋을 것 같아요. 혹시 추가 정보가 있으시면 함께 확인해 볼 수 있을 것 같습니다!감사합니다. 😊


In [9]:
# 토큰 디코딩 전략
# do_sample = False -> 가장 간단한 디코딩 방식, 가장 높은 확률을 가진 토큰 하나를 선택 -> 그리디 서치
# do_sample = True -> 샘플링 전략 사용

# 기본샘플링
import numpy as np
logits = np.array([1,2,3,4,100])

# 소프트 맥스함수 적용
from scipy.special import softmax

probas = softmax(logits)
print(probas)

[1.01122149e-43 2.74878501e-43 7.47197234e-43 2.03109266e-42
 1.00000000e+00]


In [10]:
# 마지막 원소의 확률이 거의 1에 가깝고 나머지 원소는 0에 가까움
# 확률 분포에서 토큰을 랜덤하게 선택할 때 마지막 원소가 선택

# 주어진 확률에서 샘플링 실험
np.random.multinomial(100, probas)

array([  0,   0,   0,   0, 100])

In [11]:
# 100번 모두 마지막 원소가 선택

# 확률 분포를 변형시켜 다시실행
probas = softmax(logits/100)
np.random.multinomial(100, probas)

array([13, 19, 14, 16, 38])

In [12]:
# 로짓을 1보다 큰 값으로 나누면 확률 분포가 조금 더 부드러워짐, 다른 원소가 선택될 가능성이 높아짐
# 반대로 1보다 작은 값으로 나누면 더 결정적으로 바뀌고, 가장 큰 로짓을 선택하는 그리디 서치와 비슷하게 동작
# 이렇게 선택의 다양성을 증가시키거나 줄이는 역할을 하는 값 -> 온도 파라미터

In [13]:
# temperature을 높였을 때 어떤 텍스트가 생성되는지 확인
output = pipe(messages, max_new_tokens=200, return_full_text=False, do_sample=True, temperature=10.0)
print(output[0]['generated_text'])

친절한글 안내드로 come~📗 제 응답 때문이 불편 계신다면 죄송 해요!!! 사실 해당 쿼리 바로 응대키 어려웠어서 전문 분야 종사자들 사이 피드백 공유 시간까지 거쳐 정리되는 중답니다!    Products experts 한마음 되는데 추가 준비 단계야 그래서 솔직 담백 대답 어려울까요..., 그렇지만 기대되셨다고 이해하고 빠른 응대 도움드릴 거라 굳건히 밝혀바 ~. 정확해요 추후 업데이트 드릴게요!! 이해가 궁금히셨더구?? 잠시만에 궁금하시던 대로 명확 알려도 볼까요! ❭ 😩_time delay due_.


In [14]:
# temperature 파라미터를 낮춰서 가장 큰 로짓을 가진 토큰에 높은 가능성 부여
output = pipe(messages, max_new_tokens=200, return_full_text=False, do_sample=True, temperature=0.001)
print(output[0]['generated_text'])

안녕하세요! 다이어리에 내년의 공휴일 정보가 포함되어 있는지 확인해 드리기 위해 잠시 시간이 필요합니다. 담당자가 확인 후 다시 연락드리겠습니다. 궁금한 점이 더 있으시다면 언제든지 알려주세요! 감사합니다. 😊


In [16]:
# top-k 샘플링 
# 모델이 출력한 로짓을 기준으로 최상위 k개의 토큰을 선택하는 방법, 선택된 토큰의 로짓만 소프트 맥스 함수에 통과
output = pipe(messages, max_new_tokens=200, return_full_text=False, do_sample=True, top_k=10)
print(output[0]['generated_text'])

안녕하세요! 다이어리에 내년의 공휴일 정보가 미리 표시되어 있는지에 대해 정확한 답변을 드리기 어렵습니다. 제품 개발 과정에서 다양한 요소들을 고려하기 때문에, 현재로선 확인이 필요한 부분이에요. 담당자분께서 빠르게 확인해 주실 수 있도록 시간을 좀 주세요. 감사합니다!


In [ ]:
# temperature parameter 10으로 높여 추가
output = pipe(messages, max_new_tokens=200, return_full_text=False, do_sample=True, top_k=10, temperature=10.0) 
print(output[0]['generated_text'])
# 트랜스포머 패키지는 온도파라미터를 샘플링보다 먼저 적용 후 소프트맥스함수 적용

네, 궁금하세요~! 저희가 내년 캘린더 내용은 꼼꼼 scrutinizey!(엄하게요) 제품 팀분과 빠르신 연락 부탁합니다~! 곧 답변드릴게요🎤 😄👀  자세히 알아보기 위해선 확인해 보아야 하는 부분이거덩염. 조금 기다려야 정확하신 결과가 나올 듯하답니다^^; 언제든지 질문 주셔야 빠르니 기다리겠네요.


In [18]:
# top-p 샘플링
# top-p 샘플링은 top-k 샘플링과 다르게 최상위 토큰의 개수를 고정하는 것이 아닌, 확률 순으로 토큰을 나열한 후
# 사전에 지정한 확률만큼만 최상위 토큰을 선택 ex. 상위 90%의 확률까지만 선택한다고 하면 토큰확률을 정렬후 누적된 확률이 90%일때까지만 선택
output = pipe(messages, max_new_tokens=200, return_full_text=False, do_sample=True, top_p=0.9)
print(output[0]['generated_text'])

안녕하세요! 다이어리에 내년도 공휴일 정보가 포함되어 있는지 확인하려면 바로 저희 쇼핑몰 담당자께 문의하시는 게 가장 정확할 것 같아요. 담당자분께서는 최신 정보와 제품의 세부 사항에 대해 자세히 알려드릴 수 있을 거예요. 잠시 기다리시면 바로 연락드릴게요! 감사합니다.


In [19]:
# 대상 토큰을 누적 활률로 지정하기 때문에 다양한 확률분포를 유연하게 처리가능
# but 샘플링을 위해선 소프트맥스 함수를 사용해 로짓을 확률로 바꿔줘야함, 계산량 증가

In [24]:
# top-k와 top-p샘플링 방식을 동시 적용하여 단점보완
output = pipe(messages, max_new_tokens=200, return_full_text=False, do_sample=True, top_k=120, top_p=0.9)
print(output[0]['generated_text'])

네, 저희가 확인해보니 다이어리에 내년도 공휴일이 미리 표시되어 있는지에 대해 바로 알려드리기 어렵습니다. 정확한 정보를 드리기 위해서는 제품 담당자분께 문의하시는 것이 가장 좋을 것 같습니다. 담당자분께서는 해당 다이어리의 최신 모델 정보와 정확한 기능 사항을 알려주실 수 있을 거예요. 감사합니다!
